In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llamabot import SimpleBot, StructuredBot
import frontmatter
from pyprojroot import here
from llamabot.cli.docs import UnifiedDiff, MarkdownSourceFile, documentation_information, ood_checker_sysprompt, DocumentationIssues

In [ ]:
# Open up the docs/devcontainer.md file and read YAML header
markdown_source_path = here() / "docs" / "devcontainer.md"

src_file = MarkdownSourceFile(markdown_source_path)


In [ ]:
doc_checker = StructuredBot(system_prompt=ood_checker_sysprompt(), pydantic_model=DocumentationIssues)
response = doc_checker(documentation_information(src_file, line_numbers=False, issues={}))

In [ ]:
response

In [ ]:
# For each issue in the response, generate a diff hunk against the text file
from llamabot.cli.docs import docwriter_sysprompt

In [ ]:
docwriter = SimpleBot(system_prompt=docwriter_sysprompt())
response = docwriter(documentation_information(src_file, line_numbers=False, issues=response.model_dump()))


In [ ]:
import yaml

with open(markdown_source_path, "w+") as f:
    f.write("---\n")
    # Write the intents to docs
    f.write(yaml.dump(src_file.post.metadata))
    f.write("---\n")
    f.write(response.content)